In [2]:
import requests
import time
import json
import os

api_key = 'Replace_With_Your_API'
site = 'stackoverflow'
tag = 'python'

def fetch_questions_with_answers(page, page_size):
    params = {
        'page': page,
        'pagesize': page_size,
        'order': 'desc',
        'sort': 'creation',
        'tagged': tag,
        'site': site,
        'filter': 'withbody',  # Include the body of the questions and answers
        'key': api_key
    }

    url = 'https://api.stackexchange.com/2.3/questions'
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print('Failed to retrieve data: Status code', response.status_code)
        return None

    questions = response.json().get('items', [])
    for question in questions:
        # Fetch answers for the question
        answers_response = requests.get(f"{url}/{question['question_id']}/answers", params=params)
        if answers_response.status_code == 200:
            question['answers'] = answers_response.json().get('items', [])
        else:
            question['answers'] = []

        if 'backoff' in answers_response.json():
            time.sleep(answers_response.json()['backoff'])

    return questions

def save_to_file(data, filename='Replace_With_Your_Path', mode='a'):
    # Assuming the file exists, this will open and append to it
    with open(filename, mode) as file:
        for item in data:
            json.dump(item, file)
            file.write("\n")

def main():
    print(f"Current Working Directory: {os.getcwd()}")
    page = 1 # Started from page 1
    page_size = 5  # Consider starting with a smaller page size to manage rate limits
    
    while True:
        print(f"Fetching page {page} with answers...")
        questions_with_answers = fetch_questions_with_answers(page, page_size)
        if not questions_with_answers:
            print(f"No more data found at page {page}.")
            break
        
        print(f"Fetched {len(questions_with_answers)} items.")
        save_to_file(questions_with_answers)
        
        page += 1
        time.sleep(2)  # Respect the API's rate limit

if __name__ == '__main__':
    main()

Current Working Directory: /Volumes/LaCie/LLM_Stack_Overflow
Fetching page 1666 with answers...
Failed to retrieve data: Status code 400
No more data found at page 1666.
